Here's a simple example to showcase to power of GPU parallelism in GenJAX .

In [2]:
# TODO: Vikash's example of discrete HMM with inference via checker Gibbs updates 

import genjax
from genjax import mixture_combinator, gen, normal, flip, inverse_gamma
from tensorflow_probability.substrates import jax as tfp
tfd = tfp.distributions
from jax import random
import jax.numpy as jnp

### Utility functions
class LabeledCategorical(genjax.StaticGenerativeFunction, genjax.ExactDensity):
    def sample(self, key, probs, labels, **kwargs):
        cat = tfd.Categorical(probs=probs)
        cat_index = cat.sample(seed=key)
        return labels[cat_index]

    def logpdf(self, v, probs, labels, **kwargs):
        w = jnp.log(jnp.sum(probs * (labels==v)))
        return w
    
# labcat = LabeledCategorical()

def unicat(x):
    return jnp.ones(len(x)) / len(x)
def normalize(x):
    return x / jnp.sum(x)

def discretized_gaussian(domain, mean, std):
    domain = domain.astype(jnp.float32)
    mean = mean.astype(jnp.float32)
    std = std.astype(jnp.float32)
    return normalize(
        tfd.Normal(loc=mean, scale=std).cdf(domain + .5) - 
        tfd.Normal(loc=mean, scale=std).cdf(domain - .5)
    )

# First, let's define the model
# @gen
# def init_model():
#     x = labcat(unicat(positions), positions) @ "x"
#     y = labcat(unicat(positions), positions) @ "y"
#     return (x, y)